In [1]:
### Necessary Libraries
# from msilib.schema import File
import pandas as pd, unicodedata
import string as strlab
import random, itertools
import os    
from csv import DictWriter
from datetime import date
import psycopg2
import numpy as np
from sqlalchemy import create_engine
import re 
import sys
import datetime

## Functions

In [2]:
#### Get the connection details #######################
def get_db_creds(dbname='pricedbprod',user='nextorbitprod',password='5ASyeJGXpjFFRA8k6xkj'):
    # Connect to the database
    host = "pricedbprod.c6efngmeow34.ap-south-1.rds.amazonaws.com"
    port = "5432"
    engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")
    conn = psycopg2.connect(database=dbname, user=user, password=password, host=host, port=port)
    return conn,engine
####### Get the Data from schema ##########################
def get_tabledata(schema,tablename,conn):
    query = f'SELECT * FROM {schema}."{tablename}"'
    table = pd.read_sql_query(query,conn)
    return table
########## Preprocessing the product Name ########################
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
def clean(string_word):
    if string_word:
        string_word = str(string_word)
        string_word = strip_accents(string_word)
        string_word = re.sub(r"[^ A-Za-z .:%0-9&-,-$']+", '', string_word)
        string_word = re.sub(' +', ' ', string_word)
        string_word = string_word.strip(' ')
        string_word = strlab.capwords(string_word)
        string_word = string_word.replace('1% ','')
        string_word = string_word.replace('supsup','')

        return string_word

## Location Master

In [272]:
conn,engine = get_db_creds()
Location_Data =get_tabledata(schema='restaurants',tablename='Chickfila_Location_13Apr',conn=conn)
Location_Data['brand_id']=113
Location_Data.drop(columns=['DineIn','Delivery','CarryOut','location_name','restaurant_schedules'],inplace=True)
Location_Data.rename(columns={'State':'state','DriveThru':'drivethru','CurbsidePickup':'curbsidepickup','scraped_date':'open_date'},inplace=True)
location_master = get_tabledata(schema='consolidated',tablename='location_master_20220330',conn=conn)
location_master = location_master[location_master['brand_id']!=113]
concept_master = get_tabledata(schema='consolidated',tablename='concept_master_20220330',conn=conn)
brand_master = get_tabledata(schema='consolidated',tablename='brand_master_20220330',conn=conn)
Location_Data = pd.merge(Location_Data,brand_master[['brand_id','brand_name']],on='brand_id')
Location_Data = pd.merge(Location_Data,concept_master[['brand_id','concept_name','concept_id']],on='brand_id')
Location_Data['rooftop_id']=range(max(location_master['rooftop_id'])+1,max(location_master['rooftop_id'])+1+len(Location_Data))

In [273]:
location_master = pd.concat([location_master,Location_Data])
location_master.to_sql('location_master_20220414',con=engine,schema='consolidated',if_exists='append',index=False) 

## Menu Data

### Processing Menu Data


In [3]:
conn,engine = get_db_creds()
add_on_data =  get_tabledata(schema='restaurants',tablename='Chickfila_Addons_13Apr',conn=conn)

menu_data = get_tabledata(schema='restaurants',tablename='Chickfila_Menus_13Apr',conn=conn)
menu_data = menu_data.drop_duplicates()
menu_data['brand_id']=113
menu_data['product_name']= menu_data['product_name'].apply(lambda x : clean(x))
menu_data= menu_data[~menu_data['product_name'].isin(['Gluten Free Bun'])]

### Processing Add on Data

In [4]:
add_on_data =  get_tabledata(schema='restaurants',tablename='Chickfila_Addons_13Apr',conn=conn)
add_on_data_1 = add_on_data[['source_id','base','base_price']].drop_duplicates()
add_on_data_1.rename(columns={'base':'product_name','base_price':'price'},inplace=True)
add_on_data_2=add_on_data[['source_id','add_on_name','price']].drop_duplicates()
add_on_data_2.rename(columns={'add_on_name':'product_name'},inplace=True)
add_on_data = pd.concat([add_on_data_1,add_on_data_2])
add_on_data['brand_id']=113
add_on_data['product_name']=add_on_data['product_name'].apply(lambda x :clean(x))
add_on_data['category_name']='Add on'

### Extracting Columns from Brand,Location

In [228]:
menu_data = menu_data[menu_data['price']>0]
add_on_data = add_on_data[add_on_data['price']>0]
menu_data = pd.concat([menu_data,add_on_data])
brand_master = get_tabledata(schema='consolidated',tablename='brand_master_20220330',conn=conn)
location_master = get_tabledata(schema='consolidated',tablename='location_master_20220330',conn=conn)
concept_master = get_tabledata(schema='consolidated',tablename='concept_master_20220330',conn=conn)
menu_data['source_id']=menu_data['source_id'].astype(str)
menu_data = pd.merge(menu_data,brand_master[['brand_id','brand_name']],on='brand_id')
menu_data = pd.merge(menu_data,location_master[['source_id','rooftop_id']],on='source_id')
menu_data = pd.merge(menu_data,concept_master[['brand_id','concept_name','concept_id']],on='brand_id')


In [229]:
price_data =menu_data[['source_id','brand_id','brand_name','rooftop_id','category_name','product_name','product_name_cleaned','pc_count','size','description','concept_id','concept_name','price','product_image']].drop_duplicates()
product_counts = price_data.groupby(['product_name']).agg({'source_id':'nunique'}).reset_index().rename(columns={'source_id':'Location_Count'})

## Processing on Product Master

In [230]:
product_data = price_data[['brand_id','brand_name','category_name','product_name','product_name_cleaned','pc_count','description','concept_id','concept_name','price','product_image']].drop_duplicates(subset=['product_name'],keep='first')

product_data = pd.merge(product_data,product_counts,on='product_name')
product_data['display_name']=product_data['product_name']
display_map ={**dict.fromkeys(['Small 12 Lemonade 12 Diet Lemonade','Small Hi-c Fruit Punch','Small 12 Sweet Tea 12 Unsweet Tea',"Small Barq's Root Beer",'Small Chick-fil-a Diet Lemonade','Small Chick-fil-a Lemonade','Small Coca-cola','Small Coca-cola Cherry','Small Coke Zero Sugar','Small Diet Coke','Small Diet Dr Pepper','Small Dr Pepper','Small Freshly-brewed Iced Tea Sweetened','Small Freshly-brewed Iced Tea Unsweetened','Small Mello Yello','Small Powerade','Small Sprite','Small Sunjoy (12 Sweet Tea, 12 Diet Lemonade)','Small Sunjoy (12 Sweet Tea, 12 Lemonade)','Small Sunjoy (12 Unsweet Tea, 12 Diet Lemonade)','Small Sunjoy (12 Unsweet Tea, 12 Lemonade)'],'Fountain Drink Small'),
**dict.fromkeys(['Medium 12 Lemonade 12 Diet Lemonade','Medium Hi-c Fruit Punch','Medium 12 Sweet Tea 12 Unsweet Tea',"Medium Barq's Root Beer",'Medium Chick-fil-a Diet Lemonade','Medium Chick-fil-a Lemonade','Medium Coca-cola','Medium Coca-cola Cherry','Medium Coke Zero Sugar','Medium Diet Coke','Medium Diet Dr Pepper','Medium Dr Pepper','Medium Freshly-brewed Iced Tea Sweetened','Medium Freshly-brewed Iced Tea Unsweetened','Medium Mello Yello','Medium Powerade','Medium Sprite','Medium Sunjoy (12 Sweet Tea, 12 Diet Lemonade)','Medium Sunjoy (12 Sweet Tea, 12 Lemonade)','Medium Sunjoy (12 Unsweet Tea, 12 Diet Lemonade)','Medium Sunjoy (12 Unsweet Tea, 12 Lemonade)'],'Fountain Drink Medium'),
**dict.fromkeys(['Large 12 Lemonade 12 Diet Lemonade','Large Hi-c Fruit Punch','Large 12 Sweet Tea 12 Unsweet Tea',"Large Barq's Root Beer",'Large Chick-fil-a Diet Lemonade','Large Chick-fil-a Lemonade','Large Coca-cola','Large Coca-cola Cherry','Large Coke Zero Sugar','Large Diet Coke','Large Diet Dr Pepper','Large Dr Pepper','Large Freshly-brewed Iced Tea Sweetened','Large Freshly-brewed Iced Tea Unsweetened','Large Mello Yello','Large Powerade','Large Sprite','Large Sunjoy (12 Sweet Tea, 12 Diet Lemonade)','Large Sunjoy (12 Sweet Tea, 12 Lemonade)','Large Sunjoy (12 Unsweet Tea, 12 Diet Lemonade)','Large Sunjoy (12 Unsweet Tea, 12 Lemonade)'],'Fountain Drink Large'),
**dict.fromkeys(["Small Chick-fil-a Waffle Potato Fries"],'Small Potato Fries'),
**dict.fromkeys(["Large Chick-fil-a Waffle Potato Fries"],'Large Potato Fries'),
**dict.fromkeys(["Medium Chick-fil-a Waffle Potato Fries"],'Medium Potato Fries'),
**dict.fromkeys(["White Bun (unbuttered)",'White Bun (buttered)'],'White Bun')
}


In [231]:
product_data['display_name'] = product_data['display_name'].apply(lambda x: display_map[x] if x in display_map.keys() else x)
product_data['min_calories']=''
product_data['max_calories']=''
product_data['date_modified']=pd.to_datetime('2022-04-14')
product_data['product_id']= range(113100001,113100001+(len(product_data)))
product_data['date_added']=pd.to_datetime('2022-04-14')
product_data['oz_size']=''
sort_index ={1:'Meals',2:'Entrees',3:'Sides',4:'Beverages',5:'Salads',6:'Build Your Own Family Meal',7:'Treats',8:"Kid's Meals",9:'8oz Sauces',10:'Add on'}
sort_index = dict([(value, key) for key, value in sort_index.items()])
product_data['rank']=product_data['category_name'].apply(lambda x : sort_index[x])
product_data = product_data.sort_values('rank',ascending=True)
product_data.drop(columns=['rank'],inplace=True)
product_data['menu_position']=range(1,len(product_data)+1)


In [232]:
product_data.rename(columns={'category_name':'brand_category_name','product_name':'brand_product_name','description':'brand_description'},inplace=True)
product_data=product_data[['menu_position','brand_category_name','brand_product_name','brand_description','product_name_cleaned','brand_id','date_added','date_modified','product_id','brand_name','display_name','min_calories','max_calories','pc_count','concept_id','concept_name','oz_size','price','product_image','Location_Count']]
#product_data = product_data.sort_values('menu_position',ascending=True)

In [233]:
product_data.to_csv('D:\\NextOrbit\\Projects\\ETL\\Chickfila-1\\chickfila_product_master.csv',index=False)